In [67]:
import json
sap_hana_config_file = "C:/Users/Gero/Desktop/SAP_HANA_ML/hana_cloud_config.json"

with open(sap_hana_config_file) as f:
    sap_hana_config = json.load(f)
    db_url  = sap_hana_config['url']
    db_port = sap_hana_config['port']
    db_user = sap_hana_config['user']
    db_pwd  = sap_hana_config['pwd']

In [68]:
db_port, db_user, db_url, db_pwd

(443,
 'ML_USER',
 '95822095-c1b4-4815-a43c-bdade96f9f3f.hana.trial-us10.hanacloud.ondemand.com',
 'Mineral46wasser_')

In [69]:
from hana_ml.dataframe import ConnectionContext

In [70]:
cc = ConnectionContext(db_url, db_port, db_user, db_pwd)
cc

In [8]:
print(cc.hana_version())

4.00.000.00.1686045261 (fa/CE2023.4)


In [81]:
hana_ml_df = cc.table('CUSTOMER', schema='HOTEL')
hana_ml_df

In [101]:
help(hana_ml_df.collect)

Help on method collect in module hana_ml.dataframe:

collect(fetch_size=32000, geometries=True, convert_geo_to_shapely=True) method of hana_ml.dataframe.DataFrame instance
    Copies the current DataFrame to a new Pandas DataFrame.
    
    Parameters
    ----------
    fetch_size : int, optional
        Fetch size in hdbcli.
    geometries : bool, optional
        With this flag set to `True` (default),
        the geometries are converted to Well-Known-Text representations
        in the resulting `Pandas` dataframe. Even if they are converted
        to Shapely objects (see ``convert_geo_to_shapely``), when you
        print the dataframe, the geometry columns are represented as
        Well-Known-Text.
    
        If you need the raw binary values, set this flag to `False`
    convert_geo_to_shapely : bool, optional
        If set to `True` (default), all geometry columns will be
        converted to a `Shapely` object, so that the dataframe can
        be directly used in visuali

In [18]:
df = hana_ml_df.collect()
df.head()

,CNO,TITLE,FIRSTNAME,NAME,ZIP,ADDRESS
0,3000,Mrs,Jenny,Porter,10580,"1340 N. Ash Street, #3"
1,3100,Mr,Peter,Brown,48226,"1001 34th St., APT.3"
2,3200,Company,None,Datasoft,90018,486 Maple St.
3,3300,Mrs,Rose,Brian,75243,"500 Yellowstone Drive, #2"
4,3400,Mrs,Mary,Griffith,20005,3401 Elder Lane


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CNO        14 non-null     object
 1   TITLE      14 non-null     object
 2   FIRSTNAME  12 non-null     object
 3   NAME       14 non-null     object
 4   ZIP        14 non-null     object
 5   ADDRESS    14 non-null     object
dtypes: object(6)
memory usage: 800.0+ bytes


In [14]:
df.shape

(14, 6)

# HANA ML Algorithmen

In [15]:
from hana_ml.algorithms.pal.neural_network import *

In [17]:
# Verbindung zur HANA
from hana_ml import dataframe

#connection = dataframe.ConnectionContext(KEY = 'DEV')

# CHURN laden
g_df_churn = cc.table('CHURN',
    schema = 'ML_DATA')

g_df_churn.head(20).collect()

ERROR:hana_ml.dataframe:Failed to retrieve data for the current dataframe, (362, 'invalid schema name: ML_DATA: line 1 col 37 (at pos 36)')


Error: (362, 'invalid schema name: ML_DATA: line 1 col 37 (at pos 36)')

# Walmart Pediction Dataset

In [20]:
import pandas as pd

In [29]:
df_data = pd.read_csv(
    'C:/Users/Gero/Desktop/SAP_HANA_ML/data/walmart_cleaned.csv', skipinitialspace=True, sep=',')

In [30]:
import hana_ml.dataframe as dataframe

In [89]:
conn = dataframe.ConnectionContext(db_url, db_port, db_user, db_pwd)

In [32]:
# Pandas Dataframe wird als Tabelle nach SAP Hana geladen 
# df_remote ist ein hana_ml Dataframe und beinhaltet selbst keine Daten, verweist nur auf die Tabelle Walmart
# Ziel hana_ml Dataframe ist Daten, wann immer möglich in Hana zu belassen und Berechungen auf den Daten in SAP Hana durchzuführen

df_remote = dataframe.create_dataframe_from_pandas(
            connection_context=cc,
            pandas_df=df_data,
            table_name="WALMART",
            force=True,
            drop_exist_tab=True,
            replace=False)

100%|██████████| 9/9 [02:36<00:00, 17.44s/it]


In [33]:
df_remote = conn.table("WALMART")

In [35]:
df_remote

In [36]:
df = df_remote.collect()
df.head()

,Unnamed: 0,Store,Date,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,118225,13,2010-02-19,0,48.0,1936.00,35.7,2.654,0.0,0.0,0.0,0.0,0.0,126.526286,8.316,3,219622
1,118226,13,2010-02-19,0,97.0,25077.71,35.7,2.654,0.0,0.0,0.0,0.0,0.0,126.526286,8.316,3,219622
2,118227,13,2010-02-19,0,30.0,4530.88,35.7,2.654,0.0,0.0,0.0,0.0,0.0,126.526286,8.316,3,219622
3,118228,13,2010-02-19,0,12.0,6508.70,35.7,2.654,0.0,0.0,0.0,0.0,0.0,126.526286,8.316,3,219622
4,118229,13,2010-02-19,0,9.0,31939.87,35.7,2.654,0.0,0.0,0.0,0.0,0.0,126.526286,8.316,3,219622


# Churn Modelling Dataset

In [71]:
df_data = pd.read_csv(
    'C:/Users/Gero/Desktop/SAP_HANA_ML/data/Churn_Modelling.csv', skipinitialspace=True, sep=',')

In [90]:
# Pandas Dataframe wird als Tabelle nach SAP Hana geladen 
# df_remote ist ein hana_ml Dataframe und beinhaltet selbst keine Daten, verweist nur auf die Tabelle Walmart
# Ziel hana_ml Dataframe ist Daten, wann immer möglich in Hana zu belassen und Berechungen auf den Daten in SAP Hana durchzuführen

df_remote = dataframe.create_dataframe_from_pandas(
            connection_context=cc,
            pandas_df=df_data,
            table_name="CHURN",
            force=True,
            drop_exist_tab=True,
            replace=False)

100%|██████████| 1/1 [00:03<00:00,  3.76s/it]


In [91]:
# Hana Dataframe
df_remote = conn.table("CHURN")

In [92]:
df_remote

In [93]:
# Erzeugen Pandas Dataframe
df = df_remote.collect()
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [94]:
g_df_churn = conn.table("CHURN")

In [95]:
g_df_churn

In [96]:
g_df_churn.head(20).collect()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [99]:
# 4.28
from hana_ml.algorithms.pal import partition

g_df_train, g_df_test, g_df_valid = partition.train_test_val_split(
    data = g_df_churn,
    id_column = 'CustomerId',
    partition_method = 'stratified',
    training_percentage = 0.6,
    validation_percentage = 0.0,
    testing_percentage = 0.4,
    stratified_column = 'EXITED'
    )

ERROR:hana_ml.algorithms.pal.partition:(2048, 'column store error: "ML_USER"."(DO statement)": line 29 col 1 (at pos 1222): search table error:  [34091] No ScriptServer available. See SAPNote 1650957 for further information.')
Traceback (most recent call last):
  File "C:\Users\Gero\anaconda3\lib\site-packages\hana_ml\algorithms\pal\partition.py", line 236, in train_test_val_split
    call_pal_auto_with_hint(conn,
  File "C:\Users\Gero\anaconda3\lib\site-packages\hana_ml\algorithms\pal\pal_base.py", line 1363, in call_pal_auto_with_hint
    if try_exec(cur, sql, conn):
  File "C:\Users\Gero\anaconda3\lib\site-packages\hana_ml\algorithms\pal\pal_base.py", line 1322, in try_exec
    cur.execute(sql)
hdbcli.dbapi.Error: (2048, 'column store error: "ML_USER"."(DO statement)": line 29 col 1 (at pos 1222): search table error:  [34091] No ScriptServer available. See SAPNote 1650957 for further information.')


Error: (2048, 'column store error: "ML_USER"."(DO statement)": line 29 col 1 (at pos 1222): search table error:  [34091] No ScriptServer available. See SAPNote 1650957 for further information.')

In [ ]:
# Kündigungen nach Partition zählen
g_df_train.agg([('count','CUSTOMERID','count_customer')],
    group_by = ['EXITED']).collect()

In [ ]:
# Trainingsmenge mit ausgewogener Klassenverteilung
l_df_train_0 = g_df_churn.filter("EXITED = 0").head(1000)
l_df_train_1 = g_df_churn.filter("EXITED = 1").head(1000)

In [61]:
l_df_id_train = l_df_train_0.union(l_df_train_1).select([('CUSTOMERID','ID')])
l_df_id_train_mark = l_df_id_train.select('*',('1','MARKER_TRAIN'))

ProgrammingError: (260, 'invalid column name: EXITED: line 1 col 97 (at pos 96)')

In [56]:
# Trainingsmenge mit ausgewogener Klassenverteilung
l_df_train_0 = g_df_churn.filter("EXITED = 0").head(1000)
l_df_train_1 = g_df_churn.filter("EXITED = 1").head(1000)

l_df_id_train = l_df_train_0.union(l_df_train_1).select([('CUSTOMERID','ID')])

l_df_id_train_mark = l_df_id_train.select(
    '*',('1','MARKER_TRAIN'))

# Left outer JOIN auf Grundgesamtheit
l_df_id_split = g_df_churn.alias("T").join(
    other = l_df_id_train_mark.alias('A'),
    condition = 'T.CUSTOMERID = A.ID',
    how = 'outer')

# Berechnete Spalte mit coalesce (MARKER_TRAIN, 0) 
# => nur 1 bei Kunden in Trainingsmenge
l_df_churn_split = l_df_id_split.select('*',
    ('COALESCE(MARKER_TRAIN,0)','TRAIN_SET')
    )

g_df_train_nn = l_df_churn_split.filter(
    "TRAIN_SET = 1")

g_df_test_nn = l_df_churn_split.filter(
    "TRAIN_SET = 0")

# Zählen der Kündigenden in Training- und Test
g_df_train_nn.agg([('count','EXITED','COUNT')],
    group_by = ['EXITED']).collect()

ProgrammingError: (257, 'sql syntax error: incorrect syntax near "=": line 1 col 105 (at pos 105)')